In [1]:
!pip install -q openai tavily-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 15.3 MB/s eta 0:00:00


In [2]:
import os
from google.colab import userdata

os.environ['TAVILY_API_KEY'] = userdata.get('TAVILY_API_KEY')

In [5]:
from openai import OpenAI
from tavily import TavilyClient
import os

# Define the agent's system prompt
SYSTEM_PROMPT = """
You are an intelligent agent capable of reasoning and performing actions. Follow these steps:
1. Think about the input and what needs to be done.
2. Decide on an action or answer.
3. Call functions for actions if necessary.
4. Provide a result or request additional input until the task is complete.
If an action is needed, format your response as:
"Action: <action_name>(<parameters>)"

For web searches, follow these steps:
1. Analyze the user's query and identify keywords.
2. Formulate a clear search query that captures the user's intent.
3. Call the `search_web` function with the formulated query.
4. Examine the search results and extract relevant information.
5. If necessary, refine the search query or use additional tools to gather more information.
6. Provide a final answer or summary based on the gathered information.

Examples:

User: What is the history of artificial intelligence?
Thought: I need to find information about the history of AI.
Action: search_web(query='history of artificial intelligence')
"""

# Initialize the Tavily client with your API key
tavily_client = TavilyClient()

def search_web(query):
    try:
        # Perform the search with Tavily
        response = tavily_client.search(query, max_results=3)
        # Extract and format the results
        results = response.get('results', [])
        formatted_results = "\n".join([f"{i+1}. {result['title']} - {result['url']}" for i, result in enumerate(results)])
        return formatted_results if formatted_results else "No results found."
    except Exception as e:
        return f"Error performing search: {e}"

def perform_math(operation, num1, num2):
    if operation == "add":
        return num1 + num2
    elif operation == "subtract":
        return num1 - num2
    elif operation == "multiply":
        return num1 * num2
    elif operation == "divide" and num2 != 0:
        return num1 / num2
    return "Invalid operation or parameters."

# ReAct loop implementation
def react_agent(user_input):
    conversation = [{"role": "system", "content": SYSTEM_PROMPT}]
    conversation.append({"role": "user", "content": user_input})

    client = OpenAI(api_key=userdata.get('OPENAI_API_KEY'))

    while True:
        # Generate response using OpenAI's GPT
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=conversation
        )
        message = response.choices[0].message.content
        conversation.append({"role": "assistant", "content": message})

        print("Assistant:", message)  # Show the assistant's message

        if "Thought:" in message:
            # Extract the thought process
            thought = message.split("Thought:")[1].strip()
            print(f"Thought: {thought}")  # Display the thought process

            # Continue to the next iteration to allow for Action call
            continue

        # Check for action call
        if "Action:" in message:
            action_call = message.split("Action:")[1].strip()
            action_name, params = parse_action(action_call)

            # Execute the action
            if action_name == "search_web":
                query = params.get("query", "")
                result = search_web(query)
            elif action_name == "perform_math":
                result = perform_math(
                    params.get("operation", ""),
                    params.get("num1", 0),
                    params.get("num2", 0),
                )
            else:
                result = "Unknown action."

            print(f"Observation: {result}")
            conversation.append({"role": "system", "content": f"Observation: {result}"})
        else:
            # Task is complete
            break

    return conversation[-1]["content"]

# Helper function to parse action calls
def parse_action(action_call):
    # Extract the action name and parameters
    action_name, params_str = action_call.split("(", 1)  # Limit split to 1

    print(action_name)
    print(params_str)

    action_name = action_name.strip()
    params_str = params_str.rstrip(")")
    params = {}

    if params_str:
        for param in params_str.split(","):
            if "=" in param:  # Check if key-value pair exists
                key, value = param.split("=", 1)  # Limit split to 1
                key = key.strip()
                value = value.strip().strip("'\"")
                params[key] = int(value) if value.isdigit() else value
            else:
                # Handle cases where '=' is not present, e.g., log a warning
                print(f"Warning: Parameter '{param}' is missing a key-value assignment.")
                # You can choose to ignore this parameter, assign a default value,
                # or raise an exception depending on your specific requirements.
    return action_name, params

In [13]:
if __name__ == "__main__":
    user_query = "Can you add 1553 and 2034?"
    print("Final Answer:", react_agent(user_query))

Assistant: Action: calculate_sum(1553, 2034)
Observation: Unknown action.
Assistant: The sum of 1553 and 2034 is 3587. Would you like to perform any other calculations or tasks?
Final Answer: The sum of 1553 and 2034 is 3587. Would you like to perform any other calculations or tasks?


In [6]:
if __name__ == "__main__":
    user_query = "Can you search what is a LLM in the web?"
    print("Final Answer:", react_agent(user_query))

Assistant: Action: search_web(query='what is a LLM')
search_web
query='what is a LLM')
Observation: 1. What is a large language model (LLM)? - Cloudflare - https://www.cloudflare.com/learning/ai/what-is-large-language-model/
2. What are large language models (LLMs)? - IBM - https://www.ibm.com/think/topics/large-language-models
3. What are LLMs, and how are they used in generative AI? - https://www.computerworld.com/article/1627101/what-are-large-language-models-and-how-are-they-used-in-generative-ai.html
Assistant: A Large Language Model (LLM) is a type of artificial intelligence model designed to understand and generate human-like text. These models are trained on vast amounts of text data and utilize deep learning techniques to predict the next word in a sentence, generate coherent text, and perform various natural language processing tasks.

Key features of LLMs include:

1. **Scale**: They consist of billions of parameters, making them capable of capturing complex patterns in lang